In [ ]:
""" MODELO PRE ENTRENADO SwinViT Smal
Dataset: Patch Camelyon, de tensorflow preparado para HuggingFace completo
Fuente del modelo: HuggingFace
Modelo: microsoft/swin-sbase-patch4-window7-224
e epocas
"""

' MODELO PRE ENTRENADO SwinViT Smal\nDataset: Patch Camelyon, de tensorflow preparado para HuggingFace completo\nFuente del modelo: HuggingFace\nModelo: microsoft/swin-sbase-patch4-window7-224\ne epocas\n'

In [ ]:
# Conectar con Google Drivee()
from google.colab import drive
drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"


Mounted at /content/drive


In [ ]:
# librerías
!pip install -q datasets evaluate keras_cv
!pip install --upgrade transformers
!pip install -q tensorflow
!pip install -q tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 65.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import transformers
import evaluate
from transformers import AutoModelForImageClassification, AutoImageProcessor,TrainingArguments, Trainer
from datasets import Dataset as HuggingFaceDataset, Features
from PIL import Image as PILImage
import torch

In [ ]:
# importar credenciales de Hugging face
from google.colab import userdata
import os
from huggingface_hub import login

# Accede al token almacenado en los secretos de Colab
hf_token = userdata.get('HF_TOKEN')

## Cargar el dataset patch camelyon de hugging face

In [ ]:
train_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/train")

val_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/val")

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
model_name = "microsoft/swin-base-patch4-window7-224"
batch_size = 32

## Preprocesar los datos

In [ ]:
num_labels = 2
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes = True
)

# instanciar el image processing
image_processor = AutoImageProcessor.from_pretrained(model_name,do_rescale=True,use_fast=True)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

In [ ]:
# model_path=base_folder+"/Modelos entrenados/Swin v1 Small epoc2"
# model = AutoModelForImageClassification.from_pretrained(model_path)
# image_processor = AutoImageProcessor.from_pretrained(model_path,do_rescale=True,use_fast=True)
# batch_size = 32
# num_labels = 2

In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    references = p.label_ids
    metrics_result = metric.compute(predictions=predictions, references=references)
    print(f"Metric computation result: {metrics_result}")
    return {"eval_accuracy": metrics_result["accuracy"]}

In [ ]:
# establecer los parámetros del entrenamiento
args = TrainingArguments(
    remove_unused_columns=False,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="none",
    output_dir=base_folder+"/Modelos entrenados/Swin v1 Small epoca1"
)

### Collator

In [ ]:
def custom_image_collator(batch):
    # Extract raw images and labels from the batch
    # Assuming example["image"] is already a PIL Image object
    images = [example["image"].convert("RGB") for example in batch]
    labels = [example["label"] for example in batch] # Assuming 'label' is the original label key

    # Process images using the image_processor
    processed_inputs = image_processor(images=images, return_tensors="pt")

    # Stack labels
    stacked_labels = torch.tensor(labels, dtype=torch.long)

    # Return the processed batch
    return {"pixel_values": processed_inputs["pixel_values"], "labels": stacked_labels}

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_hf_ds,
    eval_dataset=val_hf_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_image_collator,
    tokenizer=image_processor
)

/tmp/ipython-input-13-546105683.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
%%time
print("\n--- Iniciando entrenamiento ---")
train_results = trainer.train()


--- Iniciando entrenamiento ---


Epoch,Training Loss,Validation Loss,Accuracy
1,0.069400,0.316999,0.910889


Metric computation result: {'accuracy': 0.910888671875}
CPU times: user 1h 16min 39s, sys: 20min 50s, total: 1h 37min 29s
Wall time: 1h 28min 21s


In [ ]:
trainer.save_model(base_folder+"/Modelos entrenados/Swin v1 Base epoca1")

In [ ]:
# # EVALUAR EL MODELO

# # Cargar el dataset de test 32k
test_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/test")

# # Preprocesar el dataset de test
# test_hf_ds = test_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"], load_from_cache_file=False)

# # Evaluar el modelo con el dataset de test


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
%%time
test_results = trainer.evaluate(test_hf_ds)

print("\n--- Resultados de la evaluación en el dataset de test ---")
print(test_results)


Metric computation result: {'accuracy': 0.894866943359375}

--- Resultados de la evaluación en el dataset de test ---
{'eval_accuracy': 0.894866943359375, 'eval_loss': 0.33453160524368286, 'eval_runtime': 239.1931, 'eval_samples_per_second': 136.994, 'eval_steps_per_second': 4.281, 'epoch': 1.0}
CPU times: user 4min 54s, sys: 1.52 s, total: 4min 55s
Wall time: 3min 59s


### Epoca 2

In [ ]:
# establecer los parámetros del entrenamiento
args = TrainingArguments(
    remove_unused_columns=False,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="none",
    output_dir=base_folder+"/Modelos entrenados/Swin v1 Small epoca2"
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_hf_ds,
    eval_dataset=val_hf_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_image_collator,
    tokenizer=image_processor
)

/tmp/ipython-input-26-3836380531.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
%%time
print("\n--- Iniciando entrenamiento ---")
train_results = trainer.train()


--- Iniciando entrenamiento ---


Epoch,Training Loss,Validation Loss,Accuracy
1,0.049800,0.382442,0.904846


Metric computation result: {'accuracy': 0.90484619140625}
CPU times: user 1h 17min 53s, sys: 21min 40s, total: 1h 39min 34s
Wall time: 1h 28min 42s


In [ ]:
trainer.save_model(base_folder+"/Modelos entrenados/Swin v1 Base epoca2")

In [ ]:
%%time
test_results = trainer.evaluate(test_hf_ds)

print("\n--- Resultados de la evaluación en el dataset de test ---")
print(test_results)


Metric computation result: {'accuracy': 0.893402099609375}

--- Resultados de la evaluación en el dataset de test ---
{'eval_accuracy': 0.893402099609375, 'eval_loss': 0.3990659713745117, 'eval_runtime': 240.3354, 'eval_samples_per_second': 136.343, 'eval_steps_per_second': 4.261, 'epoch': 1.0}
CPU times: user 4min 57s, sys: 5.76 s, total: 5min 2s
Wall time: 4min


### Epoca 3

In [ ]:
# establecer los parámetros del entrenamiento
args = TrainingArguments(
    remove_unused_columns=False,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="none",
    output_dir=base_folder+"/Modelos entrenados/Swin v1 Small epoca3"
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_hf_ds,
    eval_dataset=val_hf_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_image_collator,
    tokenizer=image_processor
)

/tmp/ipython-input-30-1429646436.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
%%time
print("\n--- Iniciando entrenamiento ---")
train_results = trainer.train()


--- Iniciando entrenamiento ---


Epoch,Training Loss,Validation Loss,Accuracy
1,0.038700,0.503938,0.894836


Metric computation result: {'accuracy': 0.89483642578125}
CPU times: user 1h 18min 10s, sys: 19min 28s, total: 1h 37min 39s
Wall time: 1h 28min 15s


In [ ]:
trainer.save_model(base_folder+"/Modelos entrenados/Swin v1 Base epoca3")

In [ ]:
%%time
test_results = trainer.evaluate(test_hf_ds)

print("\n--- Resultados de la evaluación en el dataset de test ---")
print(test_results)


Metric computation result: {'accuracy': 0.8900146484375}

--- Resultados de la evaluación en el dataset de test ---
{'eval_accuracy': 0.8900146484375, 'eval_loss': 0.5262612104415894, 'eval_runtime': 240.8857, 'eval_samples_per_second': 136.031, 'eval_steps_per_second': 4.251, 'epoch': 1.0}
CPU times: user 4min 56s, sys: 8.51 s, total: 5min 5s
Wall time: 4min
